## iPyLeaflet
[ipyleaflet](https://github.com/ellisonbg/ipyleaflet) is a bridge between jupyter notebooks and the [leaflet](http://leafletjs.com/)  javascript library for drawing maps.

ipyleaflet comes with a few examples notebooks (this notebook was derived from one) but very little documentation,
for more documentation read the [Leaflet IPA](http://leafletjs.com/reference.html)

For installation directions, see the README on [ipyleaflet](https://github.com/ellisonbg/ipyleaflet)

[GeoJson](https://tools.ietf.org/html/rfc7946#section-3.1.7) is a good way to add data on top of an ipyleaflet map.

For other backgrounds than open streets, you can alter the ipyleaflet.py file to point to one of [these map providers](https://leaflet-extras.github.io/leaflet-providers/preview/)

In [1]:
from ipyleaflet import (
    Map,
    Marker,
    TileLayer, ImageOverlay,
    Polyline, Polygon, Rectangle, Circle, CircleMarker,
    GeoJSON,
    DrawControl
)

## Visualizing the distribution of the observations

## Load the required libraries

In [2]:
import pandas as pd
import numpy as np
import sklearn as sk
import urllib
import math
import pylab as plt

import sys
sys.path.append('./lib')

from leaflet import *

In [3]:
data_dir = "../../Data/Weather/"
from pickle import load
with open(data_dir+'/PRCP_residuals_PCA.pickle','rb') as file:
    Res_eig=load(file)
print Res_eig.keys()

#stations.pkl
with open(data_dir+'/stations.pkl','rb') as file:
    Stations=load(file)
Stations.head()


['eigen-vecs', 'stations']


,latitude,longitude,elevation,state,name,GSNFLAG,HCNFLAG,WMOID
ACW00011604,17.1167,-61.7833,10.1,NaN,ST JOHNS COOLIDGE FLD,NaN,NaN,NaN
ACW00011647,17.1333,-61.7833,19.2,NaN,ST JOHNS,NaN,NaN,NaN
AE000041196,25.3330,55.5170,34.0,NaN,SHARJAH INTER. AIRP,GSN,NaN,41196.0
AF000040930,35.3170,69.0170,3366.0,NaN,NORTH-SALANG,GSN,NaN,40930.0
AG000060390,36.7167,3.2500,24.0,NaN,ALGER-DAR EL BEIDA,GSN,NaN,60390.0


In [4]:
import pandas as pd
Eig=pd.DataFrame(Res_eig['eigen-vecs'].transpose())
Eig.head()

,0,1,2,3,4,5,6,7
0,-0.149454,-0.110416,0.024075,-0.078426,0.106084,-0.091468,0.043711,0.053613
1,-0.165481,-0.120327,0.012963,-0.091391,0.110146,-0.103869,0.061441,0.062124
2,-0.163476,-0.118427,0.022648,-0.087108,0.110017,-0.104650,0.050592,0.060052
3,-0.165315,-0.109484,0.015675,-0.089579,0.102854,-0.138707,0.080431,0.076919
4,-0.165322,-0.117057,0.023381,-0.088219,0.112662,-0.111419,0.044787,0.068507


In [5]:
Eig['station']=Res_eig['stations']
Eig=Eig.set_index('station')
print Res_eig['stations'][:5]
Eig.head()

[u'USC00192812', u'USC00193890', u'USC00192150', u'USW00014753', u'USC00191622']


,0,1,2,3,4,5,6,7
station,,,,,,,,
USC00192812,-0.149454,-0.110416,0.024075,-0.078426,0.106084,-0.091468,0.043711,0.053613
USC00193890,-0.165481,-0.120327,0.012963,-0.091391,0.110146,-0.103869,0.061441,0.062124
USC00192150,-0.163476,-0.118427,0.022648,-0.087108,0.110017,-0.104650,0.050592,0.060052
USW00014753,-0.165315,-0.109484,0.015675,-0.089579,0.102854,-0.138707,0.080431,0.076919
USC00191622,-0.165322,-0.117057,0.023381,-0.088219,0.112662,-0.111419,0.044787,0.068507


In [6]:
table=Eig.join(Stations,how='left')

In [7]:
table=table[['name','latitude','longitude','elevation',0,1,2,3,4,5,6,7]]
table.head(4)

,name,latitude,longitude,elevation,0,1,2,3,4,5,6,7
station,,,,,,,,,,,,
USC00192812,"FITCHBURG ""C""",42.5667,-71.8000,160.0,-0.149454,-0.110416,0.024075,-0.078426,0.106084,-0.091468,0.043711,0.053613
USC00193890,JAMAICA PLAIN,42.3031,-71.1239,29.0,-0.165481,-0.120327,0.012963,-0.091391,0.110146,-0.103869,0.061441,0.062124
USC00192150,EAST GLOUCESTER,42.6167,-70.6500,4.6,-0.163476,-0.118427,0.022648,-0.087108,0.110017,-0.104650,0.050592,0.060052
USW00014753,BLUE HILL,42.2122,-71.1136,190.5,-0.165315,-0.109484,0.015675,-0.089579,0.102854,-0.138707,0.080431,0.076919


In [8]:
import pylab as plt
prop_cycle = plt.rcParams['axes.prop_cycle']
colors = prop_cycle.by_key()['color']

## Map

In [11]:
min_lat,max_lat,min_long,max_long = box = (42.1103, 42.6167, -72.6, -70.8)
center = [(min_lat+max_lat)/2, (min_long+max_long)/2]
zoom = 9
m = Map(default_tiles=TileLayer(opacity=0.6), center=center, zoom=zoom)
r = Rectangle(bounds=[[min_lat,min_long],[max_lat,max_long]], weight=5, fill_opacity=0.0)
m += r

lat_margin=(max_lat-min_lat)/4
long_margin=(max_long-min_long)/4
for index,row in table.iterrows():
    _lat=row['latitude']
    _long=row['longitude']
    for col in range(4):
        _coef=row[col]
        if np.isnan(_coef):
            continue
        r=abs(_coef)/5
        #print r
        color=colors[col]
        signs=[[+1,+1],[+1,-1],[-1,-1],[-1,+1]]
        lat_sign,long_sign=signs[col]
        triangle=[(_lat,_long),(_lat+lat_sign*r,_long),(_lat,_long+long_sign*r),(_lat,_long)]
        if _coef<0:
            poly= Polygon(locations=triangle, weight=0,
                    color=color, opacity=0, fill_opacity=0.7,
                    fill_color=color)
        else:
            poly= Polygon(locations=triangle, weight=2,
                    color=color, opacity=0.8, fill_opacity=0,
                    fill_color=color)
        m +=poly   
m    

### excercises:
* Add a legend that relates the colors to values.
* Leaflet supports a variety of maps. See if you can get a topographical map as the background.

In [10]:
%matplotlib inline
pdf.plot.scatter(x='elevation',y='avg(coeff_1)');

NameError: name 'pdf' is not defined